# 3. Distinguir géneros musicales utilizando modelos de aprendizaje

En esta sección nos centraremos en entrenar un modelo basado en las características obtenidas en el notebook anterior. Con estas características trataremos de identificar si cada una de las canciones pertenecen al género clásico o no clásico.

## Importaciones

In [1]:
import torch
import torchaudio

import numpy as np

import pandas as pd
from sklearn.preprocessing import StandardScaler


____

## 3.1. Modelos de aprendizaje basados en extracción de características (*bag of songs*)

In [2]:
BATCH_SIZE = 32
LEARNING_RATE = 0.001

### 3.1.2. Creación del corpus de datos tabulares de características

En primer lugar, haremos uso de torch para crear un dataset haciendo uso de las características de audio generadas anteiormente. El uso de esta estructura nos facilitará la posterior carga de datos en un DataLoader y su uso para el entrenamiento del modelo. 

In [3]:
class TabularDataset(torch.utils.data.Dataset):

    def __init__(self, features_file, scaler=None):

        df = pd.read_csv(features_file)

        self.X = df.drop(['audio_file', 'label'], axis=1)
        self.y = df['label'].values.astype(np.int64)

        if scaler:
            self.X = scaler.transform(self.X)
        else:
            self.scaler = StandardScaler()
            self.X = self.scaler.fit_transform(self.X)

    def get_scaler(self):
        return self.scaler

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if isinstance(idx, torch.Tensor):
            idx = idx.tolist()
    
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.long)


In [4]:
ccmusic_train = TabularDataset('ccmusic/train/features.csv')
train_scaler = ccmusic_train.get_scaler()
ccmusic_train_dataloader = torch.utils.data.DataLoader(ccmusic_train, 
                                                       batch_size=BATCH_SIZE, 
                                                       shuffle=True)

ccmusic_validation = TabularDataset('ccmusic/validation/features.csv', train_scaler)
ccmusic_validation_dataloader = torch.utils.data.DataLoader(ccmusic_validation, 
                                                            batch_size=BATCH_SIZE, 
                                                            shuffle=False)

ccmusic_test = TabularDataset('ccmusic/test/features.csv', train_scaler)
ccmusic_test_dataloader = torch.utils.data.DataLoader(ccmusic_test, 
                                                      batch_size=BATCH_SIZE, 
                                                      shuffle=False)

In [5]:
ccmusic2_train = TabularDataset('ccmusic2/train/features.csv')
train_scaler2 = ccmusic2_train.get_scaler()
ccmusic2_train_dataloader = torch.utils.data.DataLoader(ccmusic2_train, 
                                                       batch_size=BATCH_SIZE, 
                                                       shuffle=True)

ccmusic2_validation = TabularDataset('ccmusic2/validation/features.csv', train_scaler2)
ccmusic2_validation_dataloader = torch.utils.data.DataLoader(ccmusic2_validation, 
                                                            batch_size=BATCH_SIZE, 
                                                            shuffle=False)

ccmusic2_test = TabularDataset('ccmusic2/test/features.csv', train_scaler2)
ccmusic2_test_dataloader = torch.utils.data.DataLoader(ccmusic2_test, 
                                                      batch_size=BATCH_SIZE, 
                                                      shuffle=False)


### 3.1.3. Definición del modelo para la clasificación de generos en base a las características

En cuanto a la elección del modelo, hemos optado por un modelo simple de perceptrón multicapa, con 3 capas lineales de 128, 64 y 1 neurona respectivamente. El restulado de esta última neurona nos indicará si la predicción corresponde a música clásica o no clásica. Como función de pérdida usaremos BinaryCrossEntropy. Con está función de loss no es necesario aplicar un función de activación sigmoide en la última capa del modelo. Además, haciendo uso del conjunto de validación, hemos aplicado la técnica de early stopping con una paciencia de 5. Con esto nos aseguramos que el modelo no se sobreajuste con los datos de entrenamiento.

In [6]:
class MLPClassifier(torch.nn.Module):

    def __init__(self, input_dim, num_classes):
        super(MLPClassifier, self).__init__()

        self.linear_block = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 128),
            torch.nn.ReLU(),
            # torch.nn.BatchNorm1d(128),
            # torch.nn.Dropout(0.5),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            # torch.nn.BatchNorm1d(64),
            torch.nn.Linear(64, num_classes if num_classes > 2 else 1)
        )

    def forward(self, x):
        return self.linear_block(x)

In [7]:
import torch
from sklearn.metrics import accuracy_score, f1_score

def train_single_epoch(model, train_dataloader, val_dataloader, loss_fn, optimizer, device):

    model.train()

    for inputs, targets in train_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        predictions = model(inputs)
        if isinstance(loss_fn, torch.nn.BCEWithLogitsLoss):
            loss = loss_fn(predictions, targets.float().unsqueeze(1))
        elif isinstance(loss_fn, torch.nn.CrossEntropyLoss):
            loss = loss_fn(predictions, targets)
        loss.backward()
        optimizer.step()  

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in val_dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            predictions = model(inputs)
            if isinstance(loss_fn, torch.nn.BCEWithLogitsLoss):
                loss = loss_fn(predictions, targets.float().unsqueeze(1))
            elif isinstance(loss_fn, torch.nn.CrossEntropyLoss):
                loss = loss_fn(predictions, targets)
            val_loss += loss.item()
    val_loss /= len(val_dataloader)

    return loss.item(), val_loss

def train(model, train_dataloader, val_dataloader, loss_fn, optimizer, epochs, patience=5, device='cuda'):

    model.to(device)
    print("Inicio del entrenamiento")

    best_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        print(f"Época {epoch+1} ", end='')
        loss, val_loss = train_single_epoch(model, train_dataloader, val_dataloader, loss_fn, optimizer, device)
        print(f"Loss: {loss}")

        if loss < best_loss:
            best_loss = loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Deteniendo entrenamiento en la época {epoch+1}")
            break

    print("Fin del entrenamiento")

Una vez entrenado el modelo, pasamos a evaluar los resultados del conjunto de test, para determinar la bondad del modelo. Debido a que el dataset se encuentra desbalanceado, calcularemos las méticas de accuracy y F1, lo que nos indicará si realmente el modelo está realizando buenas predicciones.

In [8]:
def evaluate_model(model, dataloader, num_classes, device='cuda'):
    
    model.eval()
    with torch.no_grad():
        predictions = []
        targets = []
        for inputs, target in dataloader:
            inputs, target = inputs.to(device), target.to(device)
            output = model(inputs)
            if num_classes > 2:
                output = torch.argmax(output, dim=1)
            else:
                output = torch.sigmoid(output)
                output = (output > 0.5)
            predictions.append(output)
            targets.append(target)
        predictions = torch.cat(predictions, dim=0)
        targets = torch.cat(targets, dim=0)
        return {
            'acc': accuracy_score(targets.cpu(), predictions.cpu()),
            'f1': f1_score(targets.cpu(), predictions.cpu()) if num_classes == 2 
            else f1_score(targets.cpu(), predictions.cpu(), average='micro')
        }

### 3.1.3. Entrenamiento e inferencia

#### CCMUSIC

Una vez definidas las funciones necesarias procedemos al entrenamiento del modelo. Entrenaremos duarante 50 épocas y una paciencia de 5 para early stopping.

In [ ]:
EPOCHS = 50
modelo = MLPClassifier(input_dim=ccmusic_train.X.shape[1], num_classes=2)

Podemos representar la red neuronal antes de entrenar para observar su estructura y comprobar si es necesaria alguna modificación.

In [ ]:
import netron

# Guardar el estado del modelo
torch.save(modelo.state_dict(), 'res/modelo_2tandas_weights.pth')

# Exportar a ONNX
torch.onnx.export(modelo,               # modelo instanciado
                  'models/modelo.onnx',         # donde guardar el archivo ONNX resultante
                  input_names=['input1', 'input2'],     # nombres de entrada
                  output_names=['output']               # nombres de salida
)
# Lanzar visualización
netron.start('models/modelo.onnx', browse=True)

In [ ]:
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(modelo.parameters(), 
                             lr=LEARNING_RATE)
train(modelo, ccmusic_train_dataloader, ccmusic_validation_dataloader, loss_fn, optimizer, EPOCHS)

Una vez entrenado evaluamos sobre el conjunto de test.

In [14]:
metrics_test = evaluate_model(modelo, ccmusic_test_dataloader, num_classes=2)
print(f"Accuracy en el conjunto de test: {metrics_test['acc']}")
print(f"F1 en el conjunto de test: {metrics_test['f1']}")

Accuracy en el conjunto de test: 0.9709302325581395
F1 en el conjunto de test: 0.981549815498155


Vemos que las métricas sobre test son muy buenas. Estos nos indica que, efectivamente, las características obtenidas en los pasos anteriores identifican a cada uno de los dos géneros musicales de forma efectiva. Además, cabe destacar, que estos resultados se obtienen con un conjunto de datos y modelo simples, lo que permite su entrenamiento en cualquier máquina sin ningún requisito específico.

#### CCMUSIC2

In [242]:
# # Calcular class_weight 
# import numpy as np
# import torch
# import sklearn.utils.class_weight

# # Calcula los pesos de clase usando scikit-learn
# class_weights = sklearn.utils.class_weight.compute_class_weight('balanced',
#                                                                 np.unique(ccmusic2_train.y),
#                                                                 ccmusic2_train.y)

# # Convierte los pesos de clase a tensores de PyTorch
# class_weights = torch.tensor(class_weights, dtype=torch.float32)

# print(class_weights)                    

In [17]:
EPOCHS = 100
modelo = MLPClassifier(input_dim=ccmusic2_train.X.shape[1], num_classes=9)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modelo.parameters(), 
                             lr=LEARNING_RATE )
train(modelo, ccmusic2_train_dataloader, ccmusic_validation_dataloader, loss_fn, optimizer, EPOCHS,
      patience=20)

Inicio del entrenamiento
Época 1 Loss: 3.721733808517456
Época 2 Loss: 5.6760640144348145
Época 3 Loss: 6.745384693145752
Época 4 Loss: 7.6535258293151855
Época 5 Loss: 8.097639083862305
Época 6 Loss: 8.481220245361328
Época 7 Loss: 8.62679672241211
Época 8 Loss: 9.123586654663086
Época 9 Loss: 9.292040824890137
Época 10 Loss: 9.637316703796387
Época 11 Loss: 9.899700164794922
Época 12 Loss: 10.337506294250488
Época 13 Loss: 10.493694305419922
Época 14 Loss: 10.646076202392578
Época 15 Loss: 10.925032615661621
Época 16 Loss: 10.982606887817383
Época 17 Loss: 11.610930442810059
Época 18 Loss: 11.708135604858398
Época 19 Loss: 11.9278564453125
Época 20 Loss: 11.982645034790039
Época 21 Loss: 12.389301300048828
Deteniendo entrenamiento en la época 21
Fin del entrenamiento


In [18]:
metrics_test = evaluate_model(modelo, ccmusic2_test_dataloader, num_classes=9)
print(f"Accuracy en el conjunto de test: {metrics_test['acc']}")
print(f"F1 en el conjunto de test: {metrics_test['f1']}")

Accuracy en el conjunto de test: 0.5523255813953488
F1 en el conjunto de test: 0.5523255813953488


___